In [ ]:
import cv2
from deepface import DeepFace
import time
import pygame  # pygameを使用して音を再生

# 事前に1枚の画像を読み込む
img1_path = "images/shoki01.jpg"
img1 = cv2.imread(img1_path)

# カメラを使ってリアルタイムで画像を取得する
cap = cv2.VideoCapture(0)  # カメラを開く（0はデフォルトのカメラ）

# 保存する画像のパス
saved_img_path = "images/captured_image.jpg"

# OK/NGサイン用の音声ファイルのパス
ok_sound_path = "sounds/ok_sound.mp3"  # OKサインの音 (WAV形式)
ng_sound_path = "sounds/ng_sound.mp3"  # NGサインの音 (WAV形式)

# pygameの初期化
pygame.mixer.init()

# 処理中のフラグと顔認証タイミングの制御
processing = False
show_sign_time = 0  # OK/NGサインを表示する時間
sign_duration = 3  # サインを表示する秒数
last_verified_time = time.time()  # 最後に顔認証を行った時間を記録
verification_interval = 3  # 顔認証を3秒ごとに実行
verification_result = None  # 顔認証の結果を保存

while True:
    ret, frame = cap.read()  # フレームを取得
    if not ret:
        print("カメラから映像を取得できませんでした。")
        break

    # 判定結果が出ている場合はサインを表示
    if verification_result is not None:
        # OK or NGサインを表示する時間を確認
        current_time = time.time()
        if current_time - show_sign_time < sign_duration:
            if verification_result == "OK":
                cv2.putText(frame, "OK", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            elif verification_result == "NG":
                cv2.putText(frame, "NG", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        else:
            # サインの表示時間が経過したらリセット
            verification_result = None

    # フレームを表示
    cv2.imshow("Camera", frame)

    # 'q'キーが押されたら終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 一定時間ごとに顔認証を実行する（ここでは3秒ごとに実行）
    current_time = time.time()
    if current_time - last_verified_time > verification_interval and not processing:
        processing = True
        last_verified_time = current_time  # 顔認証を行った時間を更新

        # 現在のフレームを画像として保存
        cv2.imwrite(saved_img_path, frame)
        print(f"画像を保存しました: {saved_img_path}")

        try:
            # 画像の比較を行う（保存した画像と事前に読み込んだ画像）
            print("DeepFaceで認証を実行中...")
            result = DeepFace.verify(img1_path=img1_path, img2_path=saved_img_path, enforce_detection=False)
            print(f"認証結果: {result}")

            # 判定結果が出たので、処理中フラグを解除
            processing = False

            # 同一人物かどうかの判定
            if result['verified']:
                verification_result = "OK"
                pygame.mixer.music.load(ok_sound_path)  # OKサインの音を再生
            else:
                verification_result = "NG"
                pygame.mixer.music.load(ng_sound_path)  # NGサインの音を再生

            pygame.mixer.music.play()  # 音声を再生

            # サイン表示のための時間を記録
            show_sign_time = time.time()

        except Exception as e:
            print(f"エラーが発生しました: {e}")
            processing = False

# カメラとウィンドウを解放
cap.release()
cv2.destroyAllWindows()


画像を保存しました: images/captured_image.jpg
DeepFaceで認証を実行中...
認証結果: {'verified': True, 'distance': 0.5364424015592772, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 228, 'y': 173, 'w': 221, 'h': 221, 'left_eye': None, 'right_eye': None}, 'img2': {'x': 219, 'y': 223, 'w': 182, 'h': 182, 'left_eye': None, 'right_eye': None}}, 'time': 0.58}
画像を保存しました: images/captured_image.jpg
DeepFaceで認証を実行中...
認証結果: {'verified': False, 'distance': 0.8495106661805621, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 228, 'y': 173, 'w': 221, 'h': 221, 'left_eye': None, 'right_eye': None}, 'img2': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}}, 'time': 0.55}
画像を保存しました: images/captured_image.jpg
DeepFaceで認証を実行中...
認証結果: {'verified': True, 'distance': 0.4106743786945143, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_